In [6]:
!pip install fastapi uvicorn nest-asyncio pyngrok lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831128 sha256=4eee96462d0981d733e1d2f5f48aaf87e0d4c36ad20d2b7dc67fa2900b9d1c77
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [7]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import pickle
import numpy as np
import nest_asyncio
from pyngrok import ngrok

In [8]:
nest_asyncio.apply()
app = FastAPI()

# 데이터 로딩
with open("/content/lightfm_model.pt", "rb") as f:
    model = pickle.load(f)

with open("/content/user_map.pkl", "rb") as f:
    user_map = pickle.load(f)

with open("/content/item_ids.pkl", "rb") as f:
    item_ids = pickle.load(f)

class RecommendRequest(BaseModel):
    user_id: int
    k: int = 6

@app.post("/recommend")
def recommend(req: RecommendRequest):
    user_id_raw = req.user_id
    if user_id_raw not in user_map:
        return {"recommended": []}

    user_idx = user_map[user_id_raw]
    scores = model.predict(user_idx, np.arange(len(item_ids)))
    top_items_idx = np.argsort(-scores)[:req.k]
    top_product_ids = [int(item_ids[i]) for i in top_items_idx]
    return {"recommended": top_product_ids}

In [10]:
# 복사한 토큰을 아래 YOUR_AUTHTOKEN 자리에 붙여넣으세요
!ngrok config add-authtoken 30ZZki80BWaaSqMmyq70kZKBxwc_5vJv2zxsRzdx44Kp8bbat

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# 외부에서 접근 가능한 URL을 열어줌
public_url = ngrok.connect(8000)
print(f"🌐 Public URL: {public_url}")

# uvicorn 서버 시작
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [979]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🌐 Public URL: NgrokTunnel: "https://c97ac8024fed.ngrok-free.app" -> "http://localhost:8000"
INFO:     61.75.21.155:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     61.75.21.155:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     61.75.21.155:0 - "GET /recommend HTTP/1.1" 405 Method Not Allowed
INFO:     61.75.21.155:0 - "POST /recommend HTTP/1.1" 200 OK
INFO:     61.75.21.155:0 - "POST /recommend HTTP/1.1" 200 OK


https://c97ac8024fed.ngrok-free.app/recommend
ngrok 주소